In [1]:
import numpy as np
from scipy.stats import ttest_1samp, ttest_ind, mannwhitneyu, levene, shapiro, wilcoxon
from statsmodels.stats.power import ttest_power
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [2]:
#pre and post implementation of scheme, the sales output
saleop = np.array([
[57,62],
[103,122],
[59,54],
[75,82],
[84,84],
[73,86],
[35,32],
[110,104],
[44,38],
[82,107],
[67,84],
[64,85],
[78,99],
[53,39],
[41,34],
[39,58],
[80,73],
[87,53],
[73,66],
[65,78],
[28,41],
[62,71],
[49,38],
[84,95],
[63,81],
[77,58],
[67,75],
[101,94],
[91,100],
[50,68],
])

In [3]:
# Seperating data into 2 groups
old = saleop[:, 0]
new = saleop[:, 1]
diff = new - old

### Find the mean of old scheme and new scheme column

In [6]:
old.mean()

68.03333333333333

In [7]:
new.mean()

72.03333333333333

In [8]:
new.std()

23.65795614352366

In [9]:
diff.mean()

4.0

In [10]:
diff.std()

13.844373104863458

### Use the five percent significance test over the data to determine the p-value to check new scheme has significantly raised outputs

In [11]:
# Performing shapiro and levenes test to confirm assumptions of Normality & Equal Variances

# Shapiro Test
# Null Hypothesis - Data is normally distributed
# Alternate Hypothesis - Data is not normally distributed

shapiro(old)

# p-value >= 0.05, so we fail to reject H0, so data is normally distributed

(0.9885101914405823, 0.9813658595085144)

In [12]:
shapiro(new)

# p-value >= 0.05, so we fail to reject H0, so data is normally distributed

(0.9687567353248596, 0.5057420134544373)

In [13]:
levene(old,new)
# p-value > 0.05 hence all variances are equal and Levene's test is satisfied
# this indicates that data is normally distributed

LeveneResult(statistic=1.063061539437244, pvalue=0.30679836081811235)

In [62]:
# paired t-test: doing two measurments on the same experimental unit
# e.g., before and after implementation of the scheme
t_statistic, p_value = ttest_1samp(new - old, 0)
print(t_statistic, p_value)

1.5559143823544377 0.13057553961337662


In [63]:
# null hypothesis: H0, Mu diff (the difference in mean) is equal to 0
# alternative hypothesis: Ha, Mu diff (the difference in mean) is > 0 (therefore Right Tail test will be performed)
# because the question specifically asks us to check if the new scheme has significantly raised outputs, so the
# expectation from the beginning is that output from new scheme will be > 0
# if we wanted to check for difference in mean < 0 then we would have performed Left Tail test
# if we were unsure about the outcome of the difference of the mean whther its greater or less than 0 then we would 
# have performed a two-tailed test

# Right Tail p-value is
rp = p_value / 2
print ("Right Tail t-test p-value=", rp)

# p-value obtained from two-tailed test has to be divided by 2 to give the value for Right Tail test.

Right Tail t-test p-value= 0.06528776980668831


### What conclusion does the test (p-value) lead to?

In [23]:
# If p value (for the t-test) < alpha (significance level i.e. 0.05) then we reject the H0
# if p value (for the t-test) > alpha (significance level i.e. 0.05)I fail to reject the H0
# p_value (0.06529) > 0.05 which indicates that "I fail to reject the null hypothesis H0"
# which in turn indicates that the difference in mean is quite close to 0 and thus the new scheme did not raise the 
# output levels significantly

### Suppose it has been calculated that in order for Titan to break even, the average output must increase by £5000 in the scheme compared to the old scheme. If this figure is alternative hypothesis, what is:

### The probability of a type 1 error?

In [45]:
# The probability of a Type I Error in hypothesis testing is predetermined by the significance level.
# since significance level chosen at the beginning is 5%, so probability of type 1 error is 5% as alpha = 0.05

### What is the p-value of the hypothesis test if we test for a difference of £5000?

In [50]:
# H0: Mu diff <= 5
# Ha: Mu diff > 5 
# note here the hypothesis indicate that it is a two-tailed test as a specific direction is not asked for
# paired t-test: doing two measurments on the same experimental unit
# e.g., before and after implementation of the scheme
t_statistic, p_value = ttest_1samp(new - old, 5)
print(t_statistic, p_value)

-0.3889785955886094 0.7001334912613286


In [23]:
# If p value (for the t-test) < alpha (significance level i.e. 0.05) then we reject the H0
# if p value (for the t-test) > alpha (significance level i.e. 0.05)I fail to reject the H0
# p_value = 0.70 > 0.05 which indicates that "I fail to reject the null hypothesis H0"

### Power of the test:

### Calculating p-values for Beta (Type 2 error) using T critical value

In [ ]:
# H0: Mu = 0
# Ha: Mu > 0
# we know that this is a Right tail test
# T critical for (alpha = 0.05 and degrees of freedom = 29) is 1.699 (got from T distribution table)
# we will fail to reject the null (commit a Type 2 error) if we get a T-statistic greater than +1.699
# this +1.699 T-critical value corresponds to some Xcritical value such that
# P(t-stat > +1.699) = P[X > Xcritical | Mu = 0 and s(std of sample) = 13.8]
# so we can find the Xcritical value as shown in the cell below

In [51]:
# T critical Value for a Right Tailed Test is +1.699
# Tcritical = (Xcritical - Mu (for H0))/(s/sqroot(n)) where s is standard deviation of the sample
# 1.699 = (Xcritical - 0) / (13.8/np.sqrt(30))
Xcritical = 1.699 * (13.8/np.sqrt(30))
Xcritical

4.280670875925876

In [53]:
# so I will incorrectly fail to reject the null as long as I draw a sample mean greater that 4.28
# to complete the problem what I now need to do is compute the probability of drawing a sample mean greater
# than 4.28, when given Mu = 5 and s(std of sample) = 13.8
# thus the probability of a Type 2 error at Mu = 5 is given by 
# P[X > 4.28 | Mu = 5 and s = 13.8] = P[Z > (4.28 - 5)/(13.8/np.sqrt(30))]
Tx = (4.28 - 5)/(13.8/np.sqrt(30))
Tx
# P[T > -0.286]

-0.28576829087226047

In [48]:
# pval = stats.t.sf(np.abs(tt), n-1)*2  # two-sided pvalue
# pval = stats.t.sf(np.abs(tt), n-1)    # one-sided pvalue
n = 30
pval = stats.t.sf(np.abs(-0.286), n-1)  # one-sided pvalue

print ('Beta for Type 2 error is',pval)

Beta for Type 2 error is 0.3884556128966773


In [49]:
# power of test = 1 - Beta
p = 1 - 0.388
p

0.612